# Azure AI Agents s podporou Model Context Protocol (MCP) - Python

Tento notebook demonštruje, ako používať Azure AI Agents s nástrojmi Model Context Protocol (MCP) v Pythone. Ukazuje, ako vytvoriť inteligentného agenta, ktorý dokáže využívať externé MCP servery (ako Microsoft Learn) na rozšírené funkcie pomocou autentifikácie bez kľúča.


## Inštalácia potrebných balíkov Pythonu

Najskôr musíme nainštalovať potrebné balíky Pythonu:
- **azure-ai-projects**: Základné SDK pre Azure AI Projects
- **azure-ai-agents**: SDK pre Azure AI Agents na vytváranie a správu agentov
- **azure-identity**: Poskytuje bezkľúčovú autentifikáciu pomocou DefaultAzureCredential
- **mcp**: Implementácia Model Context Protocol pre Python


## Výhody autentifikácie bez kľúčov

Tento notebook demonštruje **autentifikáciu bez kľúčov**, ktorá prináša niekoľko výhod:
- ✅ **Žiadne API kľúče na správu** - Používa autentifikáciu založenú na identite Azure
- ✅ **Zvýšená bezpečnosť** - Žiadne tajné údaje uložené v kóde alebo konfiguračných súboroch
- ✅ **Automatická rotácia poverení** - Azure sa stará o správu životného cyklu poverení
- ✅ **Riadenie prístupu na základe rolí** - Používa Azure RBAC na detailné nastavenie oprávnení
- ✅ **Podpora viacerých prostredí** - Funguje bez problémov v prostredí vývoja aj produkcie

`DefaultAzureCredential` automaticky vyberá najlepší dostupný zdroj poverení:
1. **Spravovaná identita** (pri spustení v Azure)
2. **Poverenia Azure CLI** (počas lokálneho vývoja)
3. **Poverenia Visual Studio**
4. **Premenné prostredia** (ak sú nakonfigurované)
5. **Autentifikácia cez interaktívny prehliadač** (ako záložné riešenie)


## Nastavenie autentifikácie bez kľúčov

**Predpoklady pre autentifikáciu bez kľúčov:**

### Pre lokálny vývoj:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Pre prostredia Azure:
- Aktivujte **Systémom priradenú spravovanú identitu** na vašom Azure zdroji
- Priraďte vhodné **RBAC roly** spravovanej identite:
  - `Cognitive Services OpenAI User` pre prístup k Azure OpenAI
  - `AI Developer` pre prístup k projektom Azure AI

### Premenné prostredia (voliteľné):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nie sú potrebné žiadne API kľúče ani reťazce pripojenia!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importujte potrebné knižnice

Importujte potrebné Python moduly:
- **os, time**: Štandardné Python knižnice pre premenné prostredia a oneskorenia
- **AIProjectClient**: Hlavný klient pre Azure AI Projects
- **DefaultAzureCredential**: Bezklúčová autentifikácia pre služby Azure
- **MCP-related classes**: Na vytváranie a správu MCP nástrojov a spracovanie schválení


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurácia nastavení MCP servera

Nastavte konfiguráciu MCP servera pomocou environmentálnych premenných s predvolenými hodnotami:
- **MCP_SERVER_URL**: URL adresa MCP servera (predvolene Microsoft Learn API)
- **MCP_SERVER_LABEL**: Označenie na identifikáciu MCP servera (predvolene "mslearn")

Tento prístup umožňuje flexibilnú konfiguráciu v rôznych prostrediach.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Vytvorenie klienta projektu Azure AI (Autentifikácia bez kľúča)

Inicializujte klienta projektu Azure AI pomocou **autentifikácie bez kľúča**:
- **endpoint**: URL koncového bodu projektu Azure AI Foundry
- **credential**: Používa `DefaultAzureCredential()` na bezpečnú autentifikáciu bez kľúča
- **Nie sú potrebné API kľúče**: Automaticky vyhľadáva a používa najlepšie dostupné poverenia

**Proces autentifikácie:**
1. Kontroluje Managed Identity (v prostrediach Azure)
2. Ako záložné riešenie používa poverenia Azure CLI (pre lokálny vývoj)
3. V prípade potreby využíva ďalšie dostupné zdroje poverení

Tento prístup eliminuje potrebu spravovať API kľúče alebo reťazce pripojenia vo vašom kóde.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Vytvorenie definície nástroja MCP

Vytvorte nástroj MCP, ktorý sa pripája k serveru Microsoft Learn MCP:
- **server_label**: Identifikátor pre server MCP
- **server_url**: URL koncový bod servera MCP
- **allowed_tools**: Voliteľný zoznam na obmedzenie, ktoré nástroje môžu byť použité (prázdny zoznam umožňuje všetky nástroje)

Tento nástroj umožní agentovi prístup k dokumentácii a zdrojom Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Vytvorenie agenta a vykonanie konverzácie (Práca bez kľúčov)

Táto komplexná sekcia demonštruje kompletný **pracovný postup agenta bez kľúčov**:

1. **Vytvorenie AI agenta**: Nastavte agenta s modelom GPT-4.1 nano a nástrojmi MCP
2. **Vytvorenie vlákna**: Založte konverzačné vlákno pre komunikáciu
3. **Odoslanie správy**: Opýtajte sa agenta na rozdiely medzi Azure OpenAI a OpenAI
4. **Schvaľovanie nástrojov**: Automaticky schvaľujte volania nástrojov MCP, keď je to potrebné
5. **Monitorovanie vykonávania**: Sledujte pokrok agenta a riešte potrebné akcie
6. **Zobrazenie výsledkov**: Zobrazte detaily konverzácie a použitia nástrojov

**Funkcie bez kľúčov:**
- ✅ **Žiadne pevne zakódované tajomstvá** - Všetka autentifikácia je spravovaná identitou Azure
- ✅ **Bezpečné ako štandard** - Používa kontrolu prístupu na základe rolí
- ✅ **Zjednodušené nasadenie** - Nie je potrebná správa poverení
- ✅ **Vhodné na audit** - Všetky prístupy sú sledované prostredníctvom identity Azure

Agent bude používať nástroje MCP na prístup k zdrojom Microsoft Learn s plnou bezpečnosťou a bez správy API kľúčov.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
